In [6]:
import pandas as pd
import qgrid
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import numpy as np

#  Importing economic calendar
df = pd.read_csv('EconomicCalendar.csv')
df.columns = ['Date', 'Currency', 'Impact', 'Event', 'Actual', 'Forecast','Previous']
df.Date = pd.to_datetime(df.Date)

#  Remove no and low impact rows, remove votes beacuse of #format not convertable
df = df[df.Impact != 'Non-Economic']

event_filter = ['Asset Purchase Facility Votes', 'Official Bank Rate Votes']
df = df.loc[~df['Event'].str.contains('|'.join(event_filter))]

for col in ['Actual', 'Forecast', 'Previous']:
    #  Remove rows with certain formats not convertable
    df = df.loc[~df[col].str.contains('|'.join(['\|', '\<']), na=False)]

    #  Change %, K, M, B, T into numerics
    df.loc[pd.notnull(df[col]) & df[col].str.contains('\%'), col] = \
        pd.to_numeric(df.loc[pd.notnull(df[col]) & df[col].str.contains(
            '\%'), col].str.replace('%', '')) / 100
    df.loc[pd.notnull(df[col]) & df[col].str.endswith('K'), col] = \
        pd.to_numeric(df.loc[pd.notnull(df[col]) & df[col].str.endswith(
            'K'), col].str.replace('K', '')) * 1000
    df.loc[pd.notnull(df[col]) & df[col].str.endswith('M'), col] = \
        pd.to_numeric(df.loc[pd.notnull(df[col]) & df[col].str.endswith(
            'M'), col].str.replace('M', '')) * 1000000
    df.loc[pd.notnull(df[col]) & df[col].str.endswith('B'), col] = \
        pd.to_numeric(df.loc[pd.notnull(df[col]) & df[col].str.endswith(
            'B'), col].str.replace('B', '')) * 1000000000
    df.loc[pd.notnull(df[col]) & df[col].str.endswith('T'), col] = \
        pd.to_numeric(df.loc[pd.notnull(df[col]) & df[col].str.endswith(
            'T'), col].str.replace('T', '')) * 1000000000000

    #  Change all to numeric to perform calculation
    df[col] = pd.to_numeric(df[col])

# Creating Surprise column which is Actual minus Forecast
df['Surprise'] = df['Actual'] - df['Forecast']


#  Choose impact factor and currency to show ------------------------------------------------------------>>>
impact   = ['High', 'Medium', 'Low']
currency = ['USD']

output   = df.loc[df.Currency.str.contains('|'.join(currency)) & df.Impact.str.contains('|'.join(impact))]

#  Output Variables, create bins for suprises to properly separate values of 0.001 and 10000.
surprises = output.loc[(output['Date'] >= '2018-11-01') & (output['Date'] <= '2019-04-31')& \
                        (pd.notnull(output['Surprise']))]
#  Create bins for separate plots
surprises_1 = surprises.loc[(surprises.Surprise < 0.01) & (surprises.Surprise > -0.01)]
surprises_2 = surprises.loc[((surprises.Surprise < 10) & (surprises.Surprise > -10)) & \
                            ((surprises.Surprise > 0.01) | (surprises.Surprise < -0.01))]
surprises_3 = surprises.loc[((surprises.Surprise < 100) & (surprises.Surprise > -100)) & \
                            ((surprises.Surprise > 10) | (surprises.Surprise < -10))]
surprises_4 = surprises.loc[(surprises.Surprise > 1000) | (surprises.Surprise < -1000)]

#  Creating positive and negative bins for display purposes
surprises_1_p = surprises_1[surprises_1.Surprise >= 0]
surprises_1_p_h = surprises_1_p[surprises_1_p.Impact.str.contains('High')]
surprises_1_p_m = surprises_1_p[surprises_1_p.Impact.str.contains('Medium')]
surprises_1_p_l = surprises_1_p[surprises_1_p.Impact.str.contains('Low')]

surprises_1_n = surprises_1[surprises_1.Surprise < 0]
surprises_1_n_h = surprises_1_n[surprises_1_n.Impact.str.contains('High')]
surprises_1_n_m = surprises_1_n[surprises_1_n.Impact.str.contains('Medium')]
surprises_1_n_l = surprises_1_n[surprises_1_n.Impact.str.contains('Low')]

surprises_2_p = surprises_2[surprises_2.Surprise >= 0]
surprises_2_p_h = surprises_2_p[surprises_2_p.Impact.str.contains('High')]
surprises_2_p_m = surprises_2_p[surprises_2_p.Impact.str.contains('Medium')]
surprises_2_p_l = surprises_2_p[surprises_2_p.Impact.str.contains('Low')]

surprises_2_n = surprises_2[surprises_2.Surprise < 0]
surprises_2_n_h = surprises_2_n[surprises_2_n.Impact.str.contains('High')]
surprises_2_n_m = surprises_2_n[surprises_2_n.Impact.str.contains('Medium')]
surprises_2_n_l = surprises_2_n[surprises_2_n.Impact.str.contains('Low')]

surprises_3_p = surprises_3[surprises_3.Surprise >= 0]
surprises_3_p_h = surprises_3_p[surprises_3_p.Impact.str.contains('High')]
surprises_3_p_m = surprises_3_p[surprises_3_p.Impact.str.contains('Medium')]
surprises_3_p_l = surprises_3_p[surprises_3_p.Impact.str.contains('Low')]

surprises_3_n = surprises_3[surprises_3.Surprise < 0]
surprises_3_n_h = surprises_3_n[surprises_3_n.Impact.str.contains('High')]
surprises_3_n_m = surprises_3_n[surprises_3_n.Impact.str.contains('Medium')]
surprises_3_n_l = surprises_3_n[surprises_3_n.Impact.str.contains('Low')]

surprises_4_p = surprises_4[surprises_4.Surprise >= 0]
surprises_4_p_h = surprises_4_p[surprises_4_p.Impact.str.contains('High')]
surprises_4_p_m = surprises_4_p[surprises_4_p.Impact.str.contains('Medium')]
surprises_4_p_l = surprises_4_p[surprises_4_p.Impact.str.contains('Low')]

surprises_4_n = surprises_4[surprises_4.Surprise < 0]
surprises_4_n_h = surprises_4_n[surprises_4_n.Impact.str.contains('High')]
surprises_4_n_m = surprises_4_n[surprises_4_n.Impact.str.contains('Medium')]
surprises_4_n_l = surprises_4_n[surprises_4_n.Impact.str.contains('Low')]

grouped  = output.groupby('Event')

#  Plot output
if not surprises_1.empty:
    fig, ax = plt.subplots()
    y_pos = np.arange(len(surprises_1['Event']))
    ax.barh(surprises_1_p_h['Event'], surprises_1_p_h['Surprise'], color='#153271')
    ax.barh(surprises_1_p_m['Event'], surprises_1_p_m['Surprise'], color='#3E5D9E')
    ax.barh(surprises_1_p_l['Event'], surprises_1_p_l['Surprise'], color='#8BA2D2')
    ax.barh(surprises_1_n_h['Event'], surprises_1_n_h['Surprise'], color='#DE4E21')
    ax.barh(surprises_1_n_m['Event'], surprises_1_n_m['Surprise'], color='#EC724C')
    ax.barh(surprises_1_n_l['Event'], surprises_1_n_l['Surprise'], color='#FF9A7A')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(surprises_1['Event'])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_title('One month Surprises: 1')
    plt.show()

if not surprises_2.empty:
    fig, ax = plt.subplots()
    y_pos = np.arange(len(surprises_2['Event']))
    ax.barh(surprises_2_p_h['Event'], surprises_2_p_h['Surprise'], color='#153271')
    ax.barh(surprises_2_p_m['Event'], surprises_2_p_m['Surprise'], color='#3E5D9E')
    ax.barh(surprises_2_p_l['Event'], surprises_2_p_l['Surprise'], color='#8BA2D2')
    ax.barh(surprises_2_n_h['Event'], surprises_2_n_h['Surprise'], color='#892100')
    ax.barh(surprises_2_n_m['Event'], surprises_2_n_m['Surprise'], color='#DE4E21')
    ax.barh(surprises_2_n_l['Event'], surprises_2_n_l['Surprise'], color='#FF9A7A')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(surprises_2['Event'])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_title('One month Surprises: 2')
    plt.show()

if not surprises_3.empty:    
    fig, ax = plt.subplots()
    y_pos = np.arange(len(surprises_3['Event']))
    ax.barh(surprises_3_p_h['Event'], surprises_3_p_h['Surprise'], color='#153271')
    ax.barh(surprises_3_p_m['Event'], surprises_3_p_m['Surprise'], color='#3E5D9E')
    ax.barh(surprises_3_p_l['Event'], surprises_3_p_l['Surprise'], color='#8BA2D2')
    ax.barh(surprises_3_n_h['Event'], surprises_3_n_h['Surprise'], color='#892100')
    ax.barh(surprises_3_n_m['Event'], surprises_3_n_m['Surprise'], color='#DE4E21')
    ax.barh(surprises_3_n_l['Event'], surprises_3_n_l['Surprise'], color='#FF9A7A')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(surprises_3['Event'])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_title('One month Surprises: 3')
    plt.show()

if not surprises_4.empty:    
    fig, ax = plt.subplots()
    y_pos = np.arange(len(surprises_4['Event']))
    ax.barh(surprises_4_p_h['Event'], surprises_4_p_h['Surprise'], color='#153271')
    ax.barh(surprises_4_p_m['Event'], surprises_4_p_m['Surprise'], color='#3E5D9E')
    ax.barh(surprises_4_p_l['Event'], surprises_4_p_l['Surprise'], color='#8BA2D2')
    ax.barh(surprises_4_n_h['Event'], surprises_4_n_h['Surprise'], color='#892100')
    ax.barh(surprises_4_n_m['Event'], surprises_4_n_m['Surprise'], color='#DE4E21')
    ax.barh(surprises_4_n_l['Event'], surprises_4_n_l['Surprise'], color='#FF9A7A')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(surprises_4['Event'])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_title('One month Surprises: 4')
    plt.show()

for i in grouped.groups.keys():
    x = output.loc[output.Event.str.match(i)]
    if ((not x.empty) & ( not x.Actual.isnull().all())):
        print(i) # This prints the title of the graph
        #print(output.loc[output.Event.str.match(i)][['Date','Currency', 'Impact','Event', 'Actual', 'Surprise']])
        ax = x.plot(x='Date', y=['Actual', 'Surprise'],title=i,
                                                      kind='bar', color=['#3E5D9E','#DE4E21'])
        plt.show()

ValueError: Unable to parse string "Reject" at position 488

In [ ]:
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
qgrid_widget